In [21]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [22]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [23]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [25]:
from bs4 import BeautifulSoup
import requests
import re

### 도구 정의

In [26]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [27]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [28]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

### Agent 프롬프트 작성

In [29]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [30]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

### Agent 생성

In [31]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [32]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

stream() 메소드는 에이전트의 중간 단계를 스트리밍 출력합니다. <br>

stream() 출력은 (Action, Obervation) 쌍 사이에서 번갈아가면서 나타납니다. <br>
최종적으로 에이전트가 목표를 달성했으면 답변으로 마무리됩니다. <br>

1. Action 출력
2. Observation 출력
3. Action 출력
4. Observation 출력

... 목표 달성 까지 계속 ... <br>

최종 목표가 달성되면 에이전트는 최종 답변을 출력합니다. <br>

| 단계 | 내용 |
|------|-----------|
| Action | actions : AgentAction 객체가 호출되면서 어떤 도구를 사용할지 결정을 해준다. LLM이 실행하고 실행 결과를  Obervation의 messages에 넣는다. <br> messages :  action 호출에 해당하는 채팅 메시지 |
| Obervation | steps : 현재 액션과 그 관찰을 포함한 에이전트가 지금까지 수행한 작업의 기록. <br> messages : 도구 호출의 실행 결과. 이 결과를 보고 최종 답변으로 만들게 된다. |
| Final Answer | output : Agent Finish. <br> messages : 최종 답변. |

In [33]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    print(step)
    print('==' * 50)



> Entering new None chain...
{'actions': [ToolAgentAction(tool='python_repl_tool', tool_input={'code': 'print(10 + 20)'}, log="\nInvoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_aMeQFB1BsMctyWhHmwbBvaI0', 'function': {'arguments': '{"code":"print(10 + 20)"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-4a63f542-afe7-4a80-8594-a4f2458fd405', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(10 + 20)'}, 'id': 'call_aMeQFB1BsMctyWhHmwbBvaI0', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"print(10 + 20)"}', 'id': 'call_aMeQFB1BsMctyWhHmwbBvaI0', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_aMeQFB1BsMctyWhHmwbBvaI0')], 'messages': [AIMessageChunk(content

In [34]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    print('step:', step)
    print()
    
    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    print('==' * 50)



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='python_repl_tool', tool_input={'code': 'print(10 + 20)'}, log="\nInvoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_fwsD9TmsHBr0bJSspIv6DUdH', 'function': {'arguments': '{"code":"print(10 + 20)"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-ff394251-e7d6-452d-98b4-2e8741da0d60', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(10 + 20)'}, 'id': 'call_fwsD9TmsHBr0bJSspIv6DUdH', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"print(10 + 20)"}', 'id': 'call_fwsD9TmsHBr0bJSspIv6DUdH', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_fwsD9TmsHBr0bJSspIv6DUdH')], 'messages': [AIMessageChunk(c

In [35]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    
    if 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            print("Action:")
            print(f"  Tool: {agent_step.action.tool}")
            print(f"  Tool Input: {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")
            print('==' * 50)



> Entering new None chain...

Invoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`


30
[steps]
Action:
  Tool: python_repl_tool
  Tool Input: {'code': 'print(10 + 20)'}
  Log: 
Invoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`



Observation: 30

10 + 20의 결과는 30입니다.

> Finished chain.


In [36]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    
    if 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
                
    print('==' * 50)



> Entering new None chain...

Invoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`


30
10 + 20의 결과는 30입니다.

> Finished chain.
[output]: 10 + 20의 결과는 30입니다.
Message: 10 + 20의 결과는 30입니다.


In [37]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    print('step:', step)
    print()
    print('==' * 50)

    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    elif 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            # print("Action:")
            print(f"사용 도구 이름 : {agent_step.action.tool}")
            print(f"입력 쿼리 : {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")


    elif 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
                        



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='python_repl_tool', tool_input={'code': 'print(10 + 20)'}, log="\nInvoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_5SnTYRQm01sxvk0qu23MrQJu', 'function': {'arguments': '{"code":"print(10 + 20)"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-c9f2aaf0-df04-46d2-a93b-209f0a537653', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(10 + 20)'}, 'id': 'call_5SnTYRQm01sxvk0qu23MrQJu', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"print(10 + 20)"}', 'id': 'call_5SnTYRQm01sxvk0qu23MrQJu', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_5SnTYRQm01sxvk0qu23MrQJu')], 'messages': [AIMessageChunk(c

In [38]:
result = agent_executor.stream({'input': '랭체인에 대한 뉴스를 검색해주세요'})

for step in result:
    print('step:', step)
    print()
    print('==' * 50)

    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    elif 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            # print("Action:")
            print(f"사용 도구 이름 : {agent_step.action.tool}")
            print(f"입력 쿼리 : {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")


    elif 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
        



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='search_keyword', tool_input={'query': '랭체인'}, log="\nInvoking: `search_keyword` with `{'query': '랭체인'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0xY063x27pvVglvY99hE7xO3', 'function': {'arguments': '{"query":"랭체인"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-b9c8d7ec-8530-4993-8420-46c2fba2755c', tool_calls=[{'name': 'search_keyword', 'args': {'query': '랭체인'}, 'id': 'call_0xY063x27pvVglvY99hE7xO3', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"랭체인"}', 'id': 'call_0xY063x27pvVglvY99hE7xO3', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_0xY063x27pvVglvY99hE7xO3')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'i

In [39]:
result = agent_executor.stream({'input': '"네이버 뉴스 기사의 내용을 크롤링해주세요: https://n.news.naver.com/mnews/article/002/0002372963"'})

for step in result:
    print('step:', step)
    print()
    print('==' * 50)

    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    elif 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            # print("Action:")
            print(f"사용 도구 이름 : {agent_step.action.tool}")
            print(f"입력 쿼리 : {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")


    elif 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")




> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='naver_news_crawl', tool_input={'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}, log="\nInvoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wqIHXqpWb6qCzW4unBmy2oUp', 'function': {'arguments': '{"news_url":"https://n.news.naver.com/mnews/article/002/0002372963"}', 'name': 'naver_news_crawl'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-0639f323-63f7-4fea-9f15-602102a93129', tool_calls=[{'name': 'naver_news_crawl', 'args': {'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}, 'id': 'call_wqIHXqpWb6qCzW4unBmy2oUp', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'naver_news_crawl', 'args': '{"news_url":"ht

In [40]:
result = agent_executor.stream({'input': '10 + 20 결과는?'})

for step in result:
    if 'actions' in step:   
        print('==' * 50)
        print(f'[actions]')
        print(step['actions'])
        print('==' * 50)
    elif 'steps' in step:
        print('==' * 50)
        print(f'[steps]')
        print(step['steps'])
        print('==' * 50)
    elif 'output' in step:
        print('==' * 50)
        print(f"[output]: {step['output']}")    




> Entering new None chain...
[actions]
[ToolAgentAction(tool='python_repl_tool', tool_input={'code': '10 + 20'}, log="\nInvoking: `python_repl_tool` with `{'code': '10 + 20'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_xZqXs3d4IRrvqTRuQz49liEL', 'function': {'arguments': '{"code":"10 + 20"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-f20ea24f-71ed-4d7e-a184-a7de11e9b4ed', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': '10 + 20'}, 'id': 'call_xZqXs3d4IRrvqTRuQz49liEL', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"10 + 20"}', 'id': 'call_xZqXs3d4IRrvqTRuQz49liEL', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_xZqXs3d4IRrvqTRuQz49liEL')]

Invoking: `python_repl_tool` with `{'code': '10 + 20'}`


[steps]
[Agent